In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer

pd.options.display.max_rows = 200

In [2]:
bugs = pd.read_csv('acnh_bugs.csv')
fish = pd.read_csv('acnh_fish.csv')

In [3]:
bugs['Category'] = 'Bugs'
fish['Category'] = 'Fish'

bugs.rename(columns={'Months: North Hem/South Hem':'Months'}, 
            inplace=True)
fish.rename(columns={'Months: North Hem/South Hem':'Months'}, 
            inplace=True)

In [4]:
bugs.head()

,Name,Price,Location,Time,Months,Category
0,Common Butterfly,160,Flowers,4am-7pm,Sept-Jun / Mar-Dec,Bugs
1,Yellow Butterly,160,Flying,4am-7pm,Mar-Oct / Mar-Dec,Bugs
2,Tiger Butterfly,240,Flying,4am-7pm,Mar-Sept / Sept-Mar,Bugs
3,Peacock Butterfly,2500,Rare flowers,4am-7pm,Mar-Jun / Sept-Dec,Bugs
4,Common Bluebottle,300,Flying,4am-7pm,Apr-Aug / Oct-Feb,Bugs


In [5]:
fish.head()

,Name,Price,Location,Time,Months,Category
0,Bitterling,900,River,All,Nov-Mar / May-Sept,Fish
1,Pale Chub,200,River,9am-4pm,All / All,Fish
2,Crucian Carp,160,River,All,All / All,Fish
3,Dace,240,River,4pm-9am,All / All,Fish
4,Carp,300,Pond,All,All / All,Fish


In [6]:
full_db = bugs.append(fish, sort=False)

In [7]:
full_db.head()

,Name,Price,Location,Time,Months,Category
0,Common Butterfly,160,Flowers,4am-7pm,Sept-Jun / Mar-Dec,Bugs
1,Yellow Butterly,160,Flying,4am-7pm,Mar-Oct / Mar-Dec,Bugs
2,Tiger Butterfly,240,Flying,4am-7pm,Mar-Sept / Sept-Mar,Bugs
3,Peacock Butterfly,2500,Rare flowers,4am-7pm,Mar-Jun / Sept-Dec,Bugs
4,Common Bluebottle,300,Flying,4am-7pm,Apr-Aug / Oct-Feb,Bugs


In [8]:
months_db = full_db['Months'].str.split(' ', n=-1, expand=True)

In [9]:
months_db.head()

,0,1,2
0,Sept-Jun,/,Mar-Dec
1,Mar-Oct,/,Mar-Dec
2,Mar-Sept,/,Sept-Mar
3,Mar-Jun,/,Sept-Dec
4,Apr-Aug,/,Oct-Feb


In [10]:
months_db['North_Months'] = months_db[0]
months_db['South_Months'] = months_db[2]

full_db['North_Months'] = months_db[0]
full_db['South_Months'] = months_db[2]

In [11]:
months_db.head()

,0,1,2,North_Months,South_Months
0,Sept-Jun,/,Mar-Dec,Sept-Jun,Mar-Dec
1,Mar-Oct,/,Mar-Dec,Mar-Oct,Mar-Dec
2,Mar-Sept,/,Sept-Mar,Mar-Sept,Sept-Mar
3,Mar-Jun,/,Sept-Dec,Mar-Jun,Sept-Dec
4,Apr-Aug,/,Oct-Feb,Apr-Aug,Oct-Feb


In [12]:
n_months = months_db['North_Months'].str.split('-', n=-1, expand=True)
s_months = months_db['South_Months'].str.split('-', n=-1, expand=True)

In [13]:
months = set(n_months[0])
months |= set(n_months[1])
months |= set(s_months[0])
months |= set(s_months[1])

In [14]:
print(months)

{None, 'Sept', 'Mar', 'Aug', 'June', 'Jul', 'Oct', 'July', 'May', 'March', 'Apr', 'Nov', 'All', 'Dec', 'Jun', 'Jan', 'Feb'}


In [15]:
months_dict = {'March':3, 'Feb':2, 
               'June':6, 'Apr':4, 
               'Sept':9, 'Dec':12, 
               'July':7, 'Mar':3, 
               'Nov':11, 'All':0, 
               'Oct':10, 'Jul':7, 
               'May':5, 'Jan':1, 
               'Aug':8, 'Jun':6, 
               None:0}

In [16]:
n_months['n_start'] = n_months[0].map(months_dict)
n_months['n_end'] = n_months[1].map(months_dict)
# bugs_s_months['bugs_s_start'] = bugs_s_months[0].map(months_dict)
# bugs_s_months['bugs_s_end'] = bugs_s_months[1].map(months_dict)

# fish_n_months['fish_n_start'] = fish_n_months[0].map(months_dict)
# fish_n_months['fish_n_end'] = fish_n_months[1].map(months_dict)
# fish_s_months['fish_s_start'] = fish_s_months[0].map(months_dict)
# fish_s_months['fish_s_end'] = fish_s_months[1].map(months_dict)

In [17]:
def month_range(months):
    s, e = int(months[0]), int(months[1])
    if s == 0:
        return np.arange(1, 13)
    if e == 0:
        return np.arange(s, s+1)
    if s <= e:
        return np.arange(s, e+1)
    else:
        return np.append(np.arange(1, e+1), np.arange(s, 13)) 

In [18]:
full_db['N_Months'] = n_months[['n_start', 'n_end']].values.tolist()
full_db['N_Months'] = full_db['N_Months'].apply(month_range)

# bugs['N_Months_Start'] = bugs_n_months['bugs_n_start']
# bugs['N_Months_End'] = bugs_n_months['bugs_n_end']
# bugs['S_Months_Start'] = bugs_s_months['bugs_s_start']
# bugs['S_Months_End'] = bugs_s_months['bugs_s_end']

# fish['N_Months_Start'] = fish_n_months['fish_n_start']
# fish['N_Months_End'] = fish_n_months['fish_n_end']
# fish['S_Months_Start'] = fish_s_months['fish_s_start']
# fish['S_Months_End'] = fish_s_months['fish_s_end']

In [19]:
full_db.drop(columns=['Months'], axis=1, inplace=True)

In [20]:
full_db.head()

,Name,Price,Location,Time,Category,North_Months,South_Months,N_Months
0,Common Butterfly,160,Flowers,4am-7pm,Bugs,Sept-Jun,Mar-Dec,"[1, 2, 3, 4, 5, 6, 9, 10, 11, 12]"
1,Yellow Butterly,160,Flying,4am-7pm,Bugs,Mar-Oct,Mar-Dec,"[3, 4, 5, 6, 7, 8, 9, 10]"
2,Tiger Butterfly,240,Flying,4am-7pm,Bugs,Mar-Sept,Sept-Mar,"[3, 4, 5, 6, 7, 8, 9]"
3,Peacock Butterfly,2500,Rare flowers,4am-7pm,Bugs,Mar-Jun,Sept-Dec,"[3, 4, 5, 6]"
4,Common Bluebottle,300,Flying,4am-7pm,Bugs,Apr-Aug,Oct-Feb,"[4, 5, 6, 7, 8]"


In [21]:
north_db = full_db.drop(columns=['South_Months'], axis=1)

In [22]:
mlb = MultiLabelBinarizer()

north_db = north_db.join(pd.DataFrame(mlb.fit_transform(north_db['N_Months']), columns=mlb.classes_))

In [23]:
north_db.rename(columns={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 
                         5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 
                         9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}, 
                inplace=True)

north_db.drop(columns=['North_Months', 'N_Months'], 
              axis=1, 
              inplace=True)

In [24]:
north_db.head()

,Name,Price,Location,Time,Category,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Common Butterfly,160,Flowers,4am-7pm,Bugs,1,1,1,1,1,1,0,0,1,1,1,1
0,Bitterling,900,River,All,Fish,1,1,1,1,1,1,0,0,1,1,1,1
1,Yellow Butterly,160,Flying,4am-7pm,Bugs,0,0,1,1,1,1,1,1,1,1,0,0
1,Pale Chub,200,River,9am-4pm,Fish,0,0,1,1,1,1,1,1,1,1,0,0
2,Tiger Butterfly,240,Flying,4am-7pm,Bugs,0,0,1,1,1,1,1,1,1,0,0,0


In [25]:
def time_range(time):
    if time == 'All':
        return np.arange(0, 24)
    if len(time) > 10:
        temp = time.split(' / ')
        t1, t2 = temp[0], temp[1]
        return np.append(time_rge_helper(t1), time_rge_helper(t2))
    else:
        return time_rge_helper(time)
        
def time_rge_helper(time):
    temp = time.split('-')
    for t in range(len(temp)):
        if temp[t] == '12am':
            temp[t] = 0
        elif temp[t][:-2] != 12 and temp[t][-2:] == 'pm':
            temp[t] = int(temp[t][:-2]) + 12
        else:
            temp[t] = int(temp[t][:-2])
            
    t1, t2 = temp[0], temp[1]
    if t1 <= t2:
        return np.arange(t1, t2)
    else:
        return np.append(np.arange(0, t2), np.arange(t1, 24))

In [26]:
north_db['Time'] = north_db['Time'].apply(time_range)

north_db = north_db.join(pd.DataFrame(mlb.fit_transform(north_db['Time']), columns=mlb.classes_))

In [27]:
north_db.drop(columns=['Time'], axis=1, inplace=True)

In [28]:
north_db.sort_values(by=['Price'], axis=0, inplace=True, ascending=False)

In [29]:
north_db.head()

,Name,Price,Location,Category,Jan,Feb,Mar,Apr,May,Jun,...,14,15,16,17,18,19,20,21,22,23
79,Coelacanth,15000,"Sea, raining",Fish,0,0,0,0,1,1,...,1,1,0,0,0,0,0,1,1,1
78,Barreleye,15000,Sea,Fish,1,1,1,1,1,1,...,0,0,0,0,0,1,1,1,1,1
29,Stringfish,15000,Clifftop river,Fish,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
41,Dorado,15000,River,Fish,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
28,Golden Trout,15000,Clifftop river,Fish,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [47]:
def feature_db(db, month, time):
    months={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 
            5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 
            9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
    temp = db[(db[months[month]] == 1) & (db[time] == 1)]
    temp.reset_index(inplace=True)
    
    print(str(temp.shape[0]) + " specimens")
    
    return temp[['Name', 'Price', 'Location', 'Category']]

In [48]:
feature_db(north_db, 6, 6)

58 specimens


,Name,Price,Location,Category
0,Barreleye,15000,Sea,Fish
1,Dorado,15000,River,Fish
2,Saw Shark,12000,Sea,Fish
3,Sturgeon,10000,River mouth,Fish
4,Arowana,10000,River,Fish
5,Arapaima,10000,River,Fish
6,Goliath Beetle,8000,Trees,Bugs
7,Hammerhead Shark,8000,Sea,Fish
8,Tarantula,8000,Ground,Bugs
9,Mahi-mahi,6000,Pier,Fish
